In [1]:
#paths to covariates used in model (for prediction grid)

# faosoil_id = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/angola_soil_gpkg_stuff/angola_soil_data_raster.tif"
# cropland1950 = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/hyde_lu/hyde_lu_percent/cropland_1950_percent.tif"
# annual_precip = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/bioclimatic32733_cleaned/annual_precip2.tif"
# precip_coldest_quarter = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/bioclimatic32733_cleaned/precip_coldest_quarter32733.tif"
# precip_driest_quarter = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/bioclimatic32733_cleaned/precip_driest_quarter32733.tif"
# tmax_mean = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_tmax_mean" # multiple tifs per year but only 1 value per row in final training data table
# MRRTF = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/terraincovs/MRRTF.tif"
# MRVBF = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/terraincovs/MRVBF.tif"
# aspect_cos = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/terraincovs/aspect_cos.tif"
# normalized_height = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/terraincovs/normalized_height.tif"
# relief_TRI = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/terraincovs/relief_TRI.tif"
# terrain_surf_convexity = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/terraincovs/terrain_surf_convexity.tif"
# terrain_surf_texture = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/terraincovs/terrain_surf_texture.tif"
# valley_depth = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/terraincovs/valley_depth.tif"





1) deal w tmax mean folder --> Make a single tmax_mean raster from yearly TIFFs

In [2]:
# --- Build a multi-year mean tmax raster aligned to 1 km UTM 33S ---
import os, glob
import numpy as np
import rasterio

tmax_dir = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_tmax_mean"  # folder of yearly tifs
out_tmax = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_tmax_mean_mean.tif"

files = sorted(glob.glob(os.path.join(tmax_dir, "*.tif")))
if len(files) == 0:
    raise FileNotFoundError("No yearly tmax rasters found.")

# Use the first file as template
with rasterio.open(files[0]) as src0:
    profile = src0.profile
    nodata = src0.nodata
    stack_sum = np.zeros((src0.height, src0.width), dtype=np.float64)
    stack_cnt = np.zeros_like(stack_sum, dtype=np.uint16)

for f in files:
    with rasterio.open(f) as src:
        a = src.read(1).astype(np.float64)
        if src.transform != profile['transform'] or src.crs != profile['crs'] or src.res != profile['transform'][:2]:
            raise ValueError(f"Raster alignment mismatch: {f}")
        mask = ~np.isnan(a)
        if nodata is not None:
            mask &= (a != nodata)
        stack_sum[mask] += a[mask]
        stack_cnt[mask] += 1

mean_tmax = np.where(stack_cnt > 0, stack_sum / stack_cnt, np.nan)

profile.update(dtype="float32", nodata=np.nan)

with rasterio.open(out_tmax, "w", **profile) as dst:
    dst.write(mean_tmax.astype("float32"), 1)

print(f"✅ Wrote climatological tmax: {out_tmax}")


ValueError: Raster alignment mismatch: /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_tmax_mean/tmax_1951_annual_mean.tif

## make sure all rasters align


In [ ]:
## make sure all rasters align

import rasterio, os

rasters = {
  "faosoil_id": "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/angola_soil_gpkg_stuff/angola_soil_data_raster.tif",
  "cropland_1950": "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/hyde_lu/hyde_lu_percent/cropland_1950_percent.tif",
  "annual_precip": "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/bioclimatic32733_cleaned/annual_precip2.tif",
  "precip_coldest_quarter": "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/bioclimatic32733_cleaned/precip_coldest_quarter32733.tif",
  "precip_driest_quarter": "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/bioclimatic32733_cleaned/precip_driest_quarter32733.tif",
  "tmax_mean": "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_tmax_mean_mean.tif",
  "MRRTF": "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/terraincovs/MRRTF.tif",
  "MRVBF": "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/terraincovs/MRVBF.tif",
  "aspect_cos": "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/terraincovs/aspect_cos.tif",
  "normalized_height": "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/terraincovs/normalized_height.tif",
  "relief_TRI": "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/terraincovs/relief_TRI.tif",
  "terrain_surf_convexity": "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/terraincovs/terrain_surf_convexity.tif",
  "terrain_surf_texture": "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/terraincovs/terrain_surf_texture.tif",
  "valley_depth": "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/terraincovs/valley_depth.tif",
}

tmpl = None
for name, path in rasters.items():
    with rasterio.open(path) as src:
        info = (str(src.crs), src.res, src.width, src.height, src.transform)
        if tmpl is None:
            tmpl = info
            print("Template:", name, info[:4])
        else:
            ok = (info[0] == tmpl[0]) and (info[1] == tmpl[1]) and (info[2] == tmpl[2]) and (info[3] == tmpl[3]) and (info[4] == tmpl[4])
            print(f"{name}: {'OK' if ok else 'MISMATCH'}")


## Build the CSV prediction stack (coordinates + 15 covariates)

In [ ]:
## Build the CSV prediction stack (coordinates + 15 covariates)

import rasterio
import numpy as np
import pandas as pd

# use one covariate (e.g., MRVBF) as the mask/grid driver
base_name = "MRVBF"
base_path = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/terraincovs/MRVBF.tif"

with rasterio.open(base_path) as src0:
    base = src0.read(1)
    transform = src0.transform
    nodata0 = src0.nodata
    rows, cols = np.where(~np.isnan(base) & ((nodata0 is None) | (base != nodata0)))
    xs, ys = rasterio.transform.xy(transform, rows, cols)

grid = pd.DataFrame({"X_coord": xs, "Y_coord": ys})

# add each covariate value at those grid cells
for name, path in rasters.items():
    with rasterio.open(path) as src:
        arr = src.read(1)
        vals = arr[rows, cols]
        grid[name] = vals
        print(f"Added {name}")

# optional: set categorical ints for faosoil_id
# grid['faosoil_id'] = grid['faosoil_id'].astype('Int32')

# save
out_csv = "/Users/inesschwartz/Desktop/model/covariates_stack_1km_utm33s.csv"
grid.to_csv(out_csv, index=False)
print(f"✅ Wrote stack: {out_csv} | rows={len(grid)} cols={grid.shape[1]}")


In [4]:
## working out tmax temporal issues, made a tmax_mean_mean tif instead of yearly tifs

import os, glob
import numpy as np
import rasterio

aligned_dir = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_tmax_mean_aligned"
out_tmax = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/bioclimatic32733_cleaned/tmax_mean_mean.tif"

files = sorted(glob.glob(os.path.join(aligned_dir, "*.tif")))
print(f"Found {len(files)} aligned rasters.")

with rasterio.open(files[0]) as src0:
    profile = src0.profile
    nodata = src0.nodata
    stack = np.zeros((src0.height, src0.width), dtype=np.float64)
    count = np.zeros_like(stack, dtype=np.uint16)

for f in files:
    with rasterio.open(f) as src:
        a = src.read(1).astype(np.float64)
        mask = ~np.isnan(a)
        if nodata is not None:
            mask &= (a != nodata)
        stack[mask] += a[mask]
        count[mask] += 1

mean_tmax = np.where(count > 0, stack / count, np.nan)

profile.update(dtype="float32", nodata=np.nan)

with rasterio.open(out_tmax, "w", **profile) as dst:
    dst.write(mean_tmax.astype("float32"), 1)

print(f"✅ Multi-year tmax mean written to:\n{out_tmax}")


Found 19 aligned rasters.
✅ Multi-year tmax mean written to:
/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/bioclimatic32733_cleaned/tmax_mean_mean.tif


/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_20530/2966087626.py:28: RuntimeWarning: invalid value encountered in divide
  mean_tmax = np.where(count > 0, stack / count, np.nan)


In [5]:
## do same for tmin just bc...
## working out tmax temporal issues:

import os, glob
import numpy as np
import rasterio

aligned_dir = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_tmin_mean_aligned"
out_tmax = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/bioclimatic32733_cleaned/tmin_mean_mean.tif"

files = sorted(glob.glob(os.path.join(aligned_dir, "*.tif")))
print(f"Found {len(files)} aligned rasters.")

with rasterio.open(files[0]) as src0:
    profile = src0.profile
    nodata = src0.nodata
    stack = np.zeros((src0.height, src0.width), dtype=np.float64)
    count = np.zeros_like(stack, dtype=np.uint16)

for f in files:
    with rasterio.open(f) as src:
        a = src.read(1).astype(np.float64)
        mask = ~np.isnan(a)
        if nodata is not None:
            mask &= (a != nodata)
        stack[mask] += a[mask]
        count[mask] += 1

mean_tmax = np.where(count > 0, stack / count, np.nan)

profile.update(dtype="float32", nodata=np.nan)

with rasterio.open(out_tmax, "w", **profile) as dst:
    dst.write(mean_tmax.astype("float32"), 1)

print(f"✅ Multi-year tmax mean written to:\n{out_tmax}")


Found 19 aligned rasters.
✅ Multi-year tmax mean written to:
/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/covariates_rasters/bioclimatic32733_cleaned/tmin_mean_mean.tif


/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_20530/1444607002.py:29: RuntimeWarning: invalid value encountered in divide
  mean_tmax = np.where(count > 0, stack / count, np.nan)
